In [297]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import torch
import omegaconf
import glob
import hydra
from sklearn.cluster import KMeans
from scipy.ndimage import zoom
from omegaconf import DictConfig
from omegaconf import OmegaConf
from contrastive.utils.config import process_config
from contrastive.data.datamodule import DataModule_Evaluation
from contrastive.models.contrastive_learner_fusion import \
    ContrastiveLearnerFusion

In [113]:
model_path = '/volatile/jl277509/Runs/02_STS_babies/Program/Output/ORBITAL_12-layer_k7/16-19-26_238'

In [114]:
config_path = os.path.join(model_path, '.hydra/config.yaml')
config = omegaconf.OmegaConf.load(config_path)
config = process_config(config)

config.apply_augmentations = False
config.with_labels = False

# create new models in mode visualisation
data_module = DataModule_Evaluation(config)
data_module.setup(stage='validate')

# create a new instance of the current model version,
# then load hydra weights.
print("No trained_model.pt saved. Create a new instance and load weights.")

model = ContrastiveLearnerFusion(config, sample_data=data_module)

INFO:contrastive.utils.config: Working directory : /volatile/jl277509/Runs/02_STS_babies/Program/2023_jlaval_STSbabies/contrastive/notebooks/julien/visualization
INFO:utils.py: Train/val size partitions = [18946, 2105]
INFO:utils.py: Seed for train/val split is 1
INFO:utils.py: test set size: (0, 30, 38, 22, 1)
INFO:utils.py: Length of train dataframe = 18946
INFO:utils.py: Length of val dataframe = 2105
INFO:utils.py: Length of train_val dataframe = 21051
INFO:create_datasets.py: foldlabel data NOT requested. Foldlabel data NOT loaded
INFO:create_datasets.py: distbottom data NOT requested. Distbottom data NOT loaded
INFO:contrastive_learner_fusion.py: n_datasets 1


No trained_model.pt saved. Create a new instance and load weights.


In [115]:
paths = model_path+"/logs/*/version_0/checkpoints"+r'/*.ckpt'
files = glob.glob(paths)
ckpt_path = files[0]
checkpoint = torch.load(
            ckpt_path, map_location=torch.device(config.device))
model.load_state_dict(checkpoint['state_dict'])
model.eval()

ContrastiveLearnerFusion(
  (backbones): ModuleList(
    (0): ConvNet(
      (encoder): Sequential(
        (conv0): Conv3d(1, 32, kernel_size=(7, 7, 7), stride=(1, 1, 1), padding=(3, 3, 3))
        (norm0): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (LeakyReLU0): LeakyReLU(negative_slope=0.01)
        (DropOut0): Dropout3d(p=0.05, inplace=False)
        (conv0a): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (norm0a): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (LeakyReLU0a): LeakyReLU(negative_slope=0.01)
        (DropOut0a): Dropout3d(p=0.05, inplace=False)
        (conv0b): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (norm0b): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (LeakyReLU0b): LeakyReLU(negative_slope=0.01)
        (DropOut0b): Dropout3d(p=0.05, inplace=False)
        

In [116]:
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
backbones.0.encoder.conv0.weight 	 torch.Size([32, 1, 7, 7, 7])
backbones.0.encoder.conv0.bias 	 torch.Size([32])
backbones.0.encoder.norm0.weight 	 torch.Size([32])
backbones.0.encoder.norm0.bias 	 torch.Size([32])
backbones.0.encoder.norm0.running_mean 	 torch.Size([32])
backbones.0.encoder.norm0.running_var 	 torch.Size([32])
backbones.0.encoder.norm0.num_batches_tracked 	 torch.Size([])
backbones.0.encoder.conv0a.weight 	 torch.Size([32, 32, 3, 3, 3])
backbones.0.encoder.conv0a.bias 	 torch.Size([32])
backbones.0.encoder.norm0a.weight 	 torch.Size([32])
backbones.0.encoder.norm0a.bias 	 torch.Size([32])
backbones.0.encoder.norm0a.running_mean 	 torch.Size([32])
backbones.0.encoder.norm0a.running_var 	 torch.Size([32])
backbones.0.encoder.norm0a.num_batches_tracked 	 torch.Size([])
backbones.0.encoder.conv0b.weight 	 torch.Size([32, 32, 3, 3, 3])
backbones.0.encoder.conv0b.bias 	 torch.Size([32])
backbones.0.encoder.norm0b.weight 	 torch.Size([32])
backbones.0.en

In [126]:
model = model.float()

In [25]:
# visu kernels #is it the right axis ??
kernels = model.state_dict()['backbones.0.encoder.conv0.weight'].detach()
for i in range(32):
    for k in range(7):
        kernel = kernels[i,0,k,:,:]
        plt.imshow(kernel)
        plt.savefig(f'/home/jl277509/Documents/plot_kernels/kernel_{i}_slice{k}.png')
        plt.close()

In [152]:
# load image
skeletons = np.load('/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/ORBITAL/mask/Lskeleton.npy')
skeletons = skeletons != 0

In [306]:
# find a good subject, with an interruption, visible in a plane ?
# first save the slices of this subject ? Or add to the subplot ?
subjects = pd.read_csv('/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/ORBITAL/mask/Lskeleton_subject.csv')
id = 107018
id = 104820
id = 116524
idx = subjects.loc[subjects['Subject']==id].index.tolist()[0]
data = skeletons[idx]

In [307]:
data = np.transpose(data, [3, 0, 1, 2])
data[data > 0] = 1
data = data.astype(np.float32)
data = torch.from_numpy(data)
data.unsqueeze_(0)
data.unsqueeze_(0)
print(data.shape)

torch.Size([1, 1, 1, 30, 38, 22])


In [308]:
# model.conv1 doesn't exist !!
# model.backbones[0].encoder.conv0

# For given conv layer, visu filters and voxel clusters

In [309]:
conv = 'conv0'
# NB: also need to change conv1 below ...
step_data = 1 # because of stride # conv0: 1, conv1: 2

In [310]:
# Visualize feature maps
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

model.backbones[0].encoder.conv0.register_forward_hook(get_activation(conv))
output = model(data)

act = activation[conv].squeeze()

Filters

In [311]:
nrows=4
for k in range(0, act.shape[-1]):
    fig, axs = plt.subplots(figsize=(((act.shape[0])//nrows)*4, 4*4), ncols=(act.shape[0])//nrows, nrows=nrows+1)
    for idx in range(act.shape[0]):
        axs[(idx)%nrows, (idx)//nrows].imshow(act[idx, :,:,k])
    axs[(act.shape[0]-1)%nrows+1, (act.shape[0]-1)//nrows].imshow(data[0,0,0,:,:,k*step_data])
    save_dir = f'/home/jl277509/Documents/plot_conv/{str(id)}/{conv}/'
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    plt.savefig(os.path.join(save_dir, f'{conv}_slice_{k}.png'))
    plt.close()

FileNotFoundError: [Errno 2] No such file or directory: '/home/jl277509/Documents/plot_conv/116524/conv0/'

Voxel clusters

In [312]:
n_clusters=8
kmeans = KMeans(n_clusters=n_clusters, random_state=0, n_init=1)
dim_x, dim_y, dim_z = act[0].shape

coords_list = []
reshaped_f_map = []
for x in range(dim_x):
    for y in range(dim_y):
        for z in range(dim_z):
            vector = act[:,x,y,z]
            reshaped_f_map.append(vector)
            # keep coords for reshape
            coords = (x,y,z)
            coords_list.append(coords)

reshaped_f_map = np.array(reshaped_f_map)
print(reshaped_f_map.shape)

clusters = kmeans.fit_predict(reshaped_f_map)
print(len(clusters))
print(np.unique(clusters, return_counts=True))

dim_x, dim_y, dim_z = act[0].shape

coords_list = []
reshaped_f_map = []
for x in range(dim_x):
    for y in range(dim_y):
        for z in range(dim_z):
            vector = act[:,x,y,z]
            reshaped_f_map.append(vector)
            # keep coords for reshape
            coords = (x,y,z)
            coords_list.append(coords)

reshaped_f_map = np.array(reshaped_f_map)
print(reshaped_f_map.shape)

clusters = kmeans.fit_predict(reshaped_f_map)

# Reshape to image
cluster_matrix = np.zeros((dim_x, dim_y, dim_z))
for coords, elem in zip(coords_list, clusters):
    cluster_matrix[coords]=elem

# zoom to scale the deep layers to original image shape
ratio= (skeletons.shape[1]//act.shape[1], skeletons.shape[2]//act.shape[2], skeletons.shape[3]//act.shape[3])
print(ratio)
clust = zoom(cluster_matrix, ratio, order=0)

(25080, 32)
25080
(array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int32), array([ 1875, 17387,   875,   408,   893,   723,  1862,  1057]))
(25080, 32)
(1, 1, 1)


save matrix as nifti for 2D visu with anatomist

In [ ]:
arr = cluster_matrix.astype(np.int16)
arr = np.array(arr, order='F')
vol = aims.Volume(arr)
vol.header()['voxel_size'] = [2, 2, 2]
aims.write(vol, f'/neurospin/dico/jlaval/data/clusters_visu/{str(id)}_{n_clusters}clusters_{conv}.nii.gz')

save matrix for each cluster, then make a bucket for visu

In [ ]:
for k in range(n_clusters):
    single_cluster = cluster_matrix == k
    arr = single_cluster.astype(bool)
    arr = np.array(arr, order='F')
    vol = aims.Volume(arr)
    vol.header()['voxel_size'] = [2, 2, 2]
    aims.write(vol, f'/neurospin/dico/jlaval/data/clusters_visu/binary/{str(id)}_{n_clusters}clusters_{conv}.nii.gz')

save cluster slices as png

In [296]:
## TODO: need to add a fixed colorbar across the slices !!!
for k in range(cluster_matrix.shape[-1]):
    fig, ax = plt.subplots()
    ax.imshow(cluster_matrix[:,:,k])
    save_dir = f'/home/jl277509/Documents/plot_clusters/{str(id)}/{conv}/'
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    plt.savefig(os.path.join(save_dir, f'{conv}_{n_clusters}clusters_slice_{k}.png'))
    plt.close()

In [ ]:
# Visualize conv filter # need to adapt to 3D
kernels = model.backbones[0].encoder.conv0.weight.detach()
fig, axarr = plt.subplots(kernels.size(0))
for idx in range(kernels.size(0)):
    axarr[idx].imshow(kernels[idx].squeeze())